In [23]:
import numpy as np
from sklearn import feature_extraction, decomposition

In [24]:
import sys

In [25]:
!pip install ipynb

In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
from preprocessing import *

In [28]:
preprocessing_func() 

Funkcija se pozvala


In [29]:
#U ovoj svesci koristimo Latent Dirichlet Allocation. Prije kreiranja ovog modela, morali smo pripremiti podatke tacnije pretvoriti ih u tekstove (preprocessing_func), a sada cemo jos iskoristiti CountVectorizer. 


In [30]:
import nltk
from nltk.corpus import stopwords

In [31]:
stop_words = stopwords.words('english')
stop_words
len(stop_words)

179

In [32]:
new_words=('said','says', 'say') 
for i in new_words:
    stop_words.append(i)
print(stop_words)


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [33]:
#len(stop_words)

In [34]:


documents = []
for i in range(1981, 2018):
    file = 'new_data/' + str(i) + '.npy'
    year = np.load(file)
    #print(len(year))

    for j in range(0, len(year)):
        documents.append(year[j])

        
        
#print(documents)

In [35]:
num_features = 1000


In [36]:
tf_vect = feature_extraction.text.CountVectorizer(min_df = 2, max_features=num_features, stop_words=stop_words)
tf_vect.fit(documents)
documents = tf_vect.transform(documents)

In [37]:
tf_feature_names = tf_vect.get_feature_names()
#print(tf_feature_names)

In [58]:
#LDA

#Ovdje sam provrtila vrijednosti za num_of_topics na 5, 10, 20, 25 i na kraju 50 i sacuvala
num_topics = 50

lda = decomposition.LatentDirichletAllocation(doc_topic_prior=0.2, topic_word_prior=0.2, n_components=num_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0, verbose=1).fit(documents)

iteration: 1 of max_iter: 5
iteration: 2 of max_iter: 5
iteration: 3 of max_iter: 5
iteration: 4 of max_iter: 5
iteration: 5 of max_iter: 5


In [59]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        with open('Raspodjele/50topics.txt', 'a') as f:
            print("Topic %d:" % (topic_idx), file=f)
            print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]), file=f)
            
        

no_top_words = 50
display_topics(lda, tf_feature_names, no_top_words)



In [60]:
with open('Raspodjele/50topics.txt', 'a') as f:
    print("LDA perplexity je " + str(lda.perplexity(documents)), file=f)
    print("LDA score je " + str(lda.score(documents)), file=f)


In [61]:
raspodjela = lda.transform(documents)

In [62]:
np.save("Raspodjele/50topics.npy", raspodjela)